sentiment analysis

https://spraakbanken.gu.se/resurser/absabank-imm

In [ ]:
#!pip install scikit-learn==1.0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import SGDClassifier



In [ ]:
#@title
#!wget https://svn.spraakdata.gu.se/sb-arkiv/pub/imm_absabank/datasplit.zip

In [ ]:
!unzip /content/drive/MyDrive/datasplit_sent.zip

In [ ]:
#@title
import os
import pandas as pd

def readDir(pathToDir, split = "train"):
    listOfFiles = []
    for filPath in os.listdir(pathToDir):

        if split in filPath:
          if filPath.endswith(".tsv"):
              listOfFiles.append(os.path.join(pathToDir,filPath))
    return listOfFiles


In [ ]:
list_of_files = readDir("/content/split10_consecutive_average")

In [ ]:
#@title
df_list = []

for item in list_of_files:
    df_list.append(pd.read_csv( item, delimiter = "\t"))

train_df = pd.concat(df_list,ignore_index=False)


In [ ]:
train_df.sample(3)

,doc,par,text,label
3014,www_aftonbladet_se_debatt_a_L030aQ_darfor-ar-b...,14,Men det är en grundläggande skillnad mellan at...,3.857143
2074,z01377_www_aftonbladet_se_debatt_article225783...,11,Men vi vill inte att människor tar sig till Sv...,2.000000
1789,z00698_www_svd_se_sverige-behover-en-mer-effek...,7,Sedan 2007 har 177 000 utlandsfödda fått arbet...,5.000000


In [ ]:
#@title normalize 
import math

label = []

for idx, item in train_df.iterrows():
      lab = math.ceil(item.label)
      label.append(lab)

train_df["label_N"] = label

In [ ]:
train_df.sample(5)

,doc,par,text,label,label_N
1397,z03355_www_aftonbladet_se_debatt_article102888...,9,Det vore lockande att söka amerikanskt medborg...,3.0,3
1333,z00889_flashback-59809189,11,Invandringen kostar 600 miljarder och är orsak...,1.0,1
144,z00764_flashback-62936327,5,"Jag är inte rasist, nyanlända chaufförer är of...",3.0,3
596,z03031_www_aftonbladet_se_debatt_debattamnen_p...,3,Om debattörerna syftar på personer som söker a...,3.0,3
1982,z00028_flashback-35316898,21,Vi måste stoppa massinvandringen och den antiv...,1.0,1


In [ ]:
#@title sentence model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(mod, tokenizer_name_or_path= mod , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

#Test

In [ ]:
#@title
#getting all test files

list_of_files = readDir("/content/split10_consecutive_average",split="test")

df_list = []

for item in list_of_files:
    df_list.append(pd.read_csv( item, delimiter = "\t"))

test_df = pd.concat(df_list,ignore_index=False)



In [ ]:
#@title
label = []

for idx, item in test_df.iterrows():
      lab = math.ceil(item.label)
      label.append(lab)

test_df["label_N"] = label

#EDA

In [ ]:
#@title
!git clone https://github.com/mosh98/swe_aug.git
! pip install -r /content/swe_aug/reqs.txt
!wget https://www.ida.liu.se/divisions/hcs/nlplab/swectors/swectors-300dim.txt.bz2
!bzip2 -dk /content/swectors-300dim.txt.bz2
word_vec_path = '/content/swectors-300dim.txt'
from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

# Fraction of Dataset


In [ ]:
#@title
from sklearn.model_selection import train_test_split


def frac(dataframe, fraction,label):
    """Returns fraction of data"""
    if fraction == 1.0:
      return dataframe
      
    y = dataframe[label]
    train, test = train_test_split(dataframe,train_size=fraction,stratify = y)
    
    return train

def encode_df(dataframe, col = "text"):
  encoded = []
  for idx, item in dataframe.iterrows():
         encoded.append(model.encode(item[col])) 

  return encoded


reports = [] #clf_report (Before Augment, After Augment)


In [ ]:
#@title
test_embed = []

y_test = test_df.label_N

for idx, item in test_df.iterrows():
  test_embed.append(model.encode(item.text))
  

In [ ]:
#@title
from swe_aug.Other_Techniques import Type_SR
aug_t = Type_SR.type_DA(word_vec_path)

Already downloaded a model for the 'sv' language
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title
train_df

,doc,par,text,label,label_N
0,z01627_flashback-60850068,2,All invandring är inte dålig men få kan förnek...,2.0,2
1,z01627_flashback-60850068,3,Ser man tillbaka i tiden så var det bättre på ...,2.0,2
2,z01627_flashback-60850068,4,Många blundar för problemen eller försöker med...,2.0,2
3,z01627_flashback-60850068,5,"Vi hade dock krisen 2015, sanslöst ökat våld m...",1.0,1
4,z01627_flashback-60850068,6,Så frågan är när svenskarna kommer vakna? När ...,1.0,1
...,...,...,...,...,...
3852,z02107_flashback-15247839,4,"Nu var de, enligt dem själva, kristna också = ...",3.0,3
3853,z02514_flashback-54883701,1,Professor varnar: bostäderna snart slut mestad...,2.0,2
3854,z02514_flashback-54883701,2,I SvD varnar professor Stellan Lundström att e...,2.0,2
3855,z02514_flashback-54883701,3,Att det är invandringen som spetsar till läget...,2.0,2


In [ ]:
#@title
from sklearn.linear_model import SGDClassifier

split_percentage = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
label = "label_N"

for percentage in split_percentage:
    print("______________________________________________")
    
    print("Percentage",percentage)
    
    temp_train_df = frac(train_df, percentage/100,label )
    print("Before Augmentation size: ",temp_train_df.shape)

    y_train = temp_train_df.label_N
    
    train_embed = encode_df( temp_train_df )

    logreg = SGDClassifier(max_iter=3000)
    logreg.fit(train_embed, y_train)
    
    y_pred = logreg.predict(test_embed)
    
    r_1 = classification_report(y_test, y_pred)
    print("Before Augmentation")
    print(r_1)
    print(" ")

    #Augmentation

    aug_samples = []
    for idx, item in temp_train_df.iterrows():
      txt = item.text
      lab = item.label_N
      list_of_augs = aug_t.type_synonym_sr(txt, token_type = "NOUN", n = 1)
      
      for element in list_of_augs:
          aug_samples.append({"text": ' '.join(element) ,"label_N":lab})

    
    new_aug_samples = pd.DataFrame(aug_samples)
    

    new_df = pd.concat([temp_train_df,new_aug_samples], ignore_index=True)
    
    print("After Augmentation size: ",new_df.shape[0])
    
    #augmented train test

    #encode
    train_embed_ = encode_df( new_df)
    
    y_train_ = new_df.label_N
    
    
    sgd_clf_2 = SGDClassifier(max_iter=3000)
    sgd_clf_2.fit(train_embed_, y_train_)

    y_pred = sgd_clf_2.predict(test_embed)


    r_2 = classification_report(y_test, y_pred)
    print("After Augmentation")
    print(r_2)

    #save the reports
    reports.append((percentage, r_1,r_2))



    print("______________________________________________")









______________________________________________
Percentage 10
Before Augmentation size:  (3864, 5)
Before Augmentation
              precision    recall  f1-score   support

           1       0.77      0.12      0.20       440
           2       0.48      0.74      0.58      1127
           3       0.70      0.68      0.69      2239
           4       0.60      0.58      0.59       849
           5       0.82      0.08      0.15       175

    accuracy                           0.60      4830
   macro avg       0.67      0.44      0.44      4830
weighted avg       0.64      0.60      0.58      4830

 
After Augmentation size:  7728
After Augmentation
              precision    recall  f1-score   support

           1       0.49      0.69      0.58       440
           2       0.70      0.39      0.50      1127
           3       0.71      0.70      0.71      2239
           4       0.52      0.72      0.60       849
           5       0.50      0.60      0.55       175

    accuracy   

ValueError: ignored